In [5]:
import biom
import pandas as pd
import numpy as np

In [2]:
#CLR transform
def clr(table : biom.Table):
    X = table.to_dataframe().T.values
    logX = np.log(X + 1)
    clrX = logX - logX.mean(axis=1).reshape(-1, 1)     
    return biom.Table(clrX.T, table.ids(axis='observation'), table.ids())

In [ ]:
#Qin2012T2D

table = biom.load_table("../Qin2012T2D.biom")
metadata = pd.read_csv("Qin2012T2D_metadata_new221_matched_170.txt",sep = '\t', index_col = 0)
features_to_keep = metadata.index.tolist()
table_filt = table.filter(features_to_keep, axis="sample")
with biom.util.biom_open('Qin2012T2D_170matched.biom', 'w') as f:
     table_filt.to_hdf5(f, 'counts')
        

In [6]:
#Laske2022AD
table = biom.load_table("../../../snakemake/data/Laske2022AD/Laske2022AD_filtered.biom")
new_table = clr(table)

In [7]:
new_table

3443 x 150 <class 'biom.table.Table'> with 516450 nonzero entries (100% dense)

In [8]:
table

3443 x 150 <class 'biom.table.Table'> with 381245 nonzero entries (73% dense)

In [9]:
with biom.util.biom_open('../../../snakemake/data/Laske2022AD/Laske2022AD_filtered_clr.biom', 'w') as f:
     new_table.to_hdf5(f, 'counts')

In [ ]:
#/mnt/home/djin/ceph/snakemake/data/Laske2022AD/classify
#jobid: 1985765

In [10]:
#ASD_250samples
table = biom.load_table("../../../snakemake/data/ASD/ASD_250_3studies.biom")
new_table = clr(table)

In [11]:
table

3444 x 250 <class 'biom.table.Table'> with 635665 nonzero entries (73% dense)

In [12]:
new_table

3444 x 250 <class 'biom.table.Table'> with 861000 nonzero entries (100% dense)

In [13]:
with biom.util.biom_open('../../../snakemake/data/ASD/ASD_250_3studies_clr.biom', 'w') as f:
     new_table.to_hdf5(f, 'counts')

In [19]:
#/mnt/home/djin/ceph/snakemake/data/Combined_12datasets/biom
woltka tools merge -i Laske2022AD_filtered.biom \
                   -i iMSMS2021MS_filtered.biom \
                   -i Qin2012T2D_filtered.biom \
                   -i Dan2020ASD.biom \
                   -i Wang2020ASD_filtered.biom \
                   -i Franzosa2019IBD_CD.biom \
                   -i Zhu2020Schizophrenia_filtered_new.biom \
                   -i Wan2022ASD.biom  \
                   -i Nielsen2014UC_filtered_140.biom \
                   -i Qin2010IBD_filtered_100.biom \
                   -i Qian2020PD.biom \
                   -i diabimmune.biom \
                   -o combined_12datasets.biom

# Reading profiles...
#   Read Laske2022AD_filtered.biom. Samples: 150, features: 3443.☑️
#   Read iMSMS2021MS_filtered.biom. Samples: 60, features: 3091. ☑️
#   Read Qin2012T2D_filtered.biom. Samples: 152, features: 3443. ☑️
#   Read Dan2020ASD.biom. Samples: 60, features: 3432.☑️
#   Read Wang2020ASD_filtered.biom. Samples: 62, features: 3436.☑️
#   Read Franzosa2019IBD_CD.biom. Samples: 108, features: 3442.☑️
#   Read Zhu2020Schizophrenia_filtered_new.biom. Samples: 162, features: 3443.☑️
#   Read Wan2022ASD.biom. Samples: 128, features: 3444.☑️
#   Read Nielsen2014UC_filtered_140.biom. Samples: 140, features: 3440.☑️
#   Read Qin2010IBD_filtered_100.biom. Samples: 100, features: 3338.☑️
#   Read Qian2020PD.biom. Samples: 80, features: 3438. ☑️
#   Read diabimmune.biom. Samples: 106, features: 3084.☑️
# Done. Number of profiles read: 12.
# Merging profiles... Done.
# Number of samples after merging: 1308.
# Number of features after merging: 3444.
# Merged profile written.

In [ ]:
qiime tools import \
  --input-path biom/combined_12datasets.biom \
  --type 'FeatureTable[Frequency]' \
  --input-format BIOMV210Format \
  --output-path combined_12datasets.qza


qiime sample-classifier classify-samples \
  --i-table combined_12datasets.qza \
  --m-metadata-file metadata.txt \
  --m-metadata-column Status \
  --p-optimize-feature-selection \
  --p-parameter-tuning \
  --p-estimator GradientBoostingClassifier \
  --p-n-estimators 100 \
  --p-random-state 123 \
  --output-dir GB-pictures-classifier_p100